# Feature Engineering

In this notebook, I will be creating features that will be used in my model. The features include:

- Pressure (label)
- One hot for position (edge, interior, LB, other)
- Distance to ball
- Distance to QB
- Number of players blitzing
- Pre snap distance to LOS for pass rushers

In [94]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

from scipy.stats import pearsonr

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [95]:
data = pd.read_csv("C:/Users/19012/Documents/BDB 2023/distances.csv")
data.head()

C:\Users\19012\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (47,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Unnamed: 0.1,index,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,position,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,isOnOffense,ToLeft,X_std,Y_std,back,left,right,distance_blocker,distance_rusher
0,0,177,177,2021090900,97,40151.0,6,2021-09-10T00:26:31.600,66.0,TB,right,42.10,24.02,0.51,1.98,0.06,48.28,308.34,ball_snap,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.10,24.02,36.56,30.92,16.92,NaN,1.705315
1,1,178,178,2021090900,97,40151.0,7,2021-09-10T00:26:31.700,66.0,TB,right,42.03,24.06,0.79,2.13,0.07,53.21,305.04,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.03,24.06,36.56,30.92,16.92,NaN,1.691774
2,2,179,179,2021090900,97,40151.0,8,2021-09-10T00:26:31.800,66.0,TB,right,41.95,24.10,1.00,1.84,0.09,57.07,300.02,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.95,24.10,36.56,30.92,16.92,NaN,1.647695
3,3,180,180,2021090900,97,40151.0,9,2021-09-10T00:26:31.900,66.0,TB,right,41.86,24.16,1.17,1.35,0.11,59.92,301.28,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.86,24.16,36.56,30.92,16.92,NaN,1.569841
4,4,181,181,2021090900,97,40151.0,10,2021-09-10T00:26:32.000,66.0,TB,right,41.74,24.24,1.39,1.09,0.14,62.96,301.82,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.74,24.24,36.56,30.92,16.92,NaN,1.478276


In [96]:
data.shape

(1824301, 71)

In [97]:
tracking = pd.read_csv("C:/Users/19012/Documents/BDB 2023/frame_filter_df.csv")
tracking.head()

C:\Users\19012\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,index,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,position,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,isOnOffense,ToLeft,X_std,Y_std,back,left,right
0,5,5,2021090900,97,25511.0,6,2021-09-10T00:26:31.600,12.0,TB,right,37.64,24.26,0.35,0.53,0.05,144.42,282.72,ball_snap,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,37.64,24.26,36.56,30.92,16.92
1,6,6,2021090900,97,25511.0,7,2021-09-10T00:26:31.700,12.0,TB,right,37.56,24.26,0.54,1.05,0.08,137.49,272.95,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,37.56,24.26,36.56,30.92,16.92
2,7,7,2021090900,97,25511.0,8,2021-09-10T00:26:31.800,12.0,TB,right,37.47,24.25,0.80,1.85,0.09,131.95,267.49,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,37.47,24.25,36.56,30.92,16.92
3,8,8,2021090900,97,25511.0,9,2021-09-10T00:26:31.900,12.0,TB,right,37.38,24.24,0.99,2.03,0.09,129.85,263.48,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,37.38,24.24,36.56,30.92,16.92
4,9,9,2021090900,97,25511.0,10,2021-09-10T00:26:32.000,12.0,TB,right,37.27,24.23,1.19,1.82,0.11,123.79,263.77,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,37.27,24.23,36.56,30.92,16.92


In [98]:
tracking.shape

(4361846, 68)

## Pressure (label)

Create the label we will be using as the outcome for the model.

In [99]:
data['pff_hit'].unique()

array([nan,  0.,  1.])

In [100]:
data['pressure'] = np.where((data['pff_hit']==1) | (data['pff_hurry']==1) | (data['pff_sack']==1), 1,0)

## Pressure Allowed (label)

Create the label we will be using as the outcome for the pressure allowed model.

In [101]:
data['pressure_allowed'] = np.where((data['pff_hitAllowed']==1) | (data['pff_hurryAllowed']==1) | (data['pff_sackAllowed']==1), 1,0)

## One Hot Encode Position Group

For the pass rushers, 3 columns will be created. 'edge' = 1 will be an edge rusher, 'interior' = 1 will be an interior DL, and 'LB' = 1 will be a linebacker. If all 3 columns = 0, then this will indicate another position blitzling (ex. CB, S).

For pass blockers, 4 columns will be created. 'T' = 1 will be a tackle, 'G' = 1 will be a guard, 'C' = 1 will be a center, and 'interiorOL' = 1 will be an interior OL. If all 4 columns = 0, then this will indicate another position pass blocking (ex. RB, TE, etc.).

This encoding is completed to account for a position in the model because we don't want to compare players against the average rusher/blocker, we want to compare players to the average player at the respective position. For example, any blitzing LB is at a disadvantage when compared to Myles Garrett because the latter is built to handle OL. Therefore, LB will consistently have low xPress measures. We want to compare LBs to other LBs to see how much better they are than the average LB and not the average pass rusher.

In [102]:
# pass rushers
data['edge'] = np.where(data['position']=='DE',1,0)
data['interiorDL'] = np.where((data['position']=='DT') | (data['position']=='NT'),1,0)
data['LB'] = np.where((data['position']=='LB'),1,0)

# pass blockers
data['T'] = np.where((data['position']=='T'),1,0)
data['G'] = np.where((data['position']=='G'),1,0)
data['C'] = np.where((data['position']=='C'),1,0)
data['interiorOL'] = np.where((data['position']=='C') | (data['position']=='G'),1,0)

data.head()

,Unnamed: 0,Unnamed: 0.1,index,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,position,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,isOnOffense,ToLeft,X_std,Y_std,back,left,right,distance_blocker,distance_rusher,pressure,pressure_allowed,edge,interiorDL,LB,T,G,C,interiorOL
0,0,177,177,2021090900,97,40151.0,6,2021-09-10T00:26:31.600,66.0,TB,right,42.10,24.02,0.51,1.98,0.06,48.28,308.34,ball_snap,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.10,24.02,36.56,30.92,16.92,NaN,1.705315,0,0,0,0,0,0,0,1,1
1,1,178,178,2021090900,97,40151.0,7,2021-09-10T00:26:31.700,66.0,TB,right,42.03,24.06,0.79,2.13,0.07,53.21,305.04,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.03,24.06,36.56,30.92,16.92,NaN,1.691774,0,0,0,0,0,0,0,1,1
2,2,179,179,2021090900,97,40151.0,8,2021-09-10T00:26:31.800,66.0,TB,right,41.95,24.10,1.00,1.84,0.09,57.07,300.02,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.95,24.10,36.56,30.92,16.92,NaN,1.647695,0,0,0,0,0,0,0,1,1
3,3,180,180,2021090900,97,40151.0,9,2021-09-10T00:26:31.900,66.0,TB,right,41.86,24.16,1.17,1.35,0.11,59.92,301.28,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.86,24.16,36.56,30.92,16.92,NaN,1.569841,0,0,0,0,0,0,0,1,1
4,4,181,181,2021090900,97,40151.0,10,2021-09-10T00:26:32.000,66.0,TB,right,41.74,24.24,1.39,1.09,0.14,62.96,301.82,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.74,24.24,36.56,30.92,16.92,NaN,1.478276,0,0,0,0,0,0,0,1,1


## Number of Players Blitzing

For this feature, group by game/play/frame and count the number of pff_role=='Pass Rush'.

In [103]:
data['pff_role'].unique()

array(['Pass Block', 'Pass Rush'], dtype=object)

In [104]:
pass_rushers = data[data['pff_role']=='Pass Rush'].groupby(['gameId','playId','frameId']).count()[['pff_role']]
pass_rushers = pass_rushers.reset_index()
pass_rushers = pass_rushers.rename(columns={'pff_role': 'num_rushers'})

In [105]:
pass_rushers.head()

,gameId,playId,frameId,num_rushers
0,2021090900,97,6,5
1,2021090900,97,7,5
2,2021090900,97,8,5
3,2021090900,97,9,5
4,2021090900,97,10,5


In [106]:
data = data.merge(pass_rushers, on=['gameId','playId','frameId'], how='left')
# sanity check (1824301)
data.shape

(1824301, 81)

In [107]:
data.head()

,Unnamed: 0,Unnamed: 0.1,index,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,position,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,isOnOffense,ToLeft,X_std,Y_std,back,left,right,distance_blocker,distance_rusher,pressure,pressure_allowed,edge,interiorDL,LB,T,G,C,interiorOL,num_rushers
0,0,177,177,2021090900,97,40151.0,6,2021-09-10T00:26:31.600,66.0,TB,right,42.10,24.02,0.51,1.98,0.06,48.28,308.34,ball_snap,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.10,24.02,36.56,30.92,16.92,NaN,1.705315,0,0,0,0,0,0,0,1,1,5.0
1,1,178,178,2021090900,97,40151.0,7,2021-09-10T00:26:31.700,66.0,TB,right,42.03,24.06,0.79,2.13,0.07,53.21,305.04,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.03,24.06,36.56,30.92,16.92,NaN,1.691774,0,0,0,0,0,0,0,1,1,5.0
2,2,179,179,2021090900,97,40151.0,8,2021-09-10T00:26:31.800,66.0,TB,right,41.95,24.10,1.00,1.84,0.09,57.07,300.02,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.95,24.10,36.56,30.92,16.92,NaN,1.647695,0,0,0,0,0,0,0,1,1,5.0
3,3,180,180,2021090900,97,40151.0,9,2021-09-10T00:26:31.900,66.0,TB,right,41.86,24.16,1.17,1.35,0.11,59.92,301.28,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.86,24.16,36.56,30.92,16.92,NaN,1.569841,0,0,0,0,0,0,0,1,1,5.0
4,4,181,181,2021090900,97,40151.0,10,2021-09-10T00:26:32.000,66.0,TB,right,41.74,24.24,1.39,1.09,0.14,62.96,301.82,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.74,24.24,36.56,30.92,16.92,NaN,1.478276,0,0,0,0,0,0,0,1,1,5.0


## Presnap Distance to LOS

To find the distance a defensive player is to the LOS at the time of snap, I will filter the data frame by event=='ball_snap' and get the y location of the ball.

In [108]:
los = tracking[(tracking['event']=='ball_snap') & (tracking['team']=='football')][['gameId','playId','x']]
los = los.rename(columns={'x': 'los_snap'})
los.head()

,gameId,playId,los_snap
770,2021090900,97,41.56
1377,2021090900,137,108.68
1909,2021090900,187,78.21
2658,2021090900,282,50.19
3285,2021090900,349,54.87


In [109]:
data = data.merge(los,on=['gameId','playId'],how='left')
# sanity check (1824301)
data.shape

(1824301, 82)

In [110]:
data[data['los_snap'].isna()].shape

(178, 82)

178 frames out of 1824301 frames don't have a football it looks like. That's less than 1% so we can just remove those plays.

In [111]:
data = data[~data['los_snap'].isna()]

In [112]:
dist_to_los_snap = data[data['event']=='ball_snap']
dist_to_los_snap['dist_los_snap'] = dist_to_los_snap['los_snap'] - dist_to_los_snap['X_std']
dist_to_los_snap = dist_to_los_snap[['gameId','playId','nflId','dist_los_snap']]

<ipython-input-112-ecdaa3920b9b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_to_los_snap['dist_los_snap'] = dist_to_los_snap['los_snap'] - dist_to_los_snap['X_std']


In [113]:
dist_to_los_snap.head()

,gameId,playId,nflId,dist_los_snap
0,2021090900,97,40151.0,-0.54
35,2021090900,97,41263.0,-1.74
70,2021090900,97,42377.0,0.18
105,2021090900,97,42403.0,-2.34
140,2021090900,97,42404.0,-0.37


In [114]:
data = data.merge(dist_to_los_snap,on=['gameId','playId','nflId'],how='left')
# sanity check (1824123)
data.shape

(1824123, 83)

In [115]:
data.head()

,Unnamed: 0,Unnamed: 0.1,index,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,position,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,isOnOffense,ToLeft,X_std,Y_std,back,left,right,distance_blocker,distance_rusher,pressure,pressure_allowed,edge,interiorDL,LB,T,G,C,interiorOL,num_rushers,los_snap,dist_los_snap
0,0,177,177,2021090900,97,40151.0,6,2021-09-10T00:26:31.600,66.0,TB,right,42.10,24.02,0.51,1.98,0.06,48.28,308.34,ball_snap,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.10,24.02,36.56,30.92,16.92,NaN,1.705315,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54
1,1,178,178,2021090900,97,40151.0,7,2021-09-10T00:26:31.700,66.0,TB,right,42.03,24.06,0.79,2.13,0.07,53.21,305.04,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.03,24.06,36.56,30.92,16.92,NaN,1.691774,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54
2,2,179,179,2021090900,97,40151.0,8,2021-09-10T00:26:31.800,66.0,TB,right,41.95,24.10,1.00,1.84,0.09,57.07,300.02,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.95,24.10,36.56,30.92,16.92,NaN,1.647695,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54
3,3,180,180,2021090900,97,40151.0,9,2021-09-10T00:26:31.900,66.0,TB,right,41.86,24.16,1.17,1.35,0.11,59.92,301.28,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.86,24.16,36.56,30.92,16.92,NaN,1.569841,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54
4,4,181,181,2021090900,97,40151.0,10,2021-09-10T00:26:32.000,66.0,TB,right,41.74,24.24,1.39,1.09,0.14,62.96,301.82,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.74,24.24,36.56,30.92,16.92,NaN,1.478276,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54


## Fix Orientation

We standardized the x- and y-coordinates to have every play go the same direction; however, we neglected that this would affect the orientation.

https://www.kaggle.com/code/statsbymichaellopez/nfl-tracking-wrangling-voronoi-and-sonars/notebook

In [116]:
data['O_std_1'] = np.where((data['ToLeft']==True) & (data['o'] < 90),data['o']+360, data['o'])
data['O_std_1'] = np.where((data['ToLeft']==True) & (data['o'] > 270), data['o']-360, data['O_std_1'])

data['O_std_2'] = np.where((data['ToLeft']==True) & (data['o'] > 270), data['O_std_1']-180, data['O_std_1'])

## Fix Direction

In [117]:
data['Dir_std_1'] = np.where((data['ToLeft']==True) & (data['dir'] < 90),data['dir']+360, data['dir'])
data['Dir_std_1'] = np.where((data['ToLeft']==True) & (data['dir'] > 270), data['dir']-360, data['Dir_std_1'])

data['Dir_std_2'] = np.where((data['ToLeft']==True) & (data['dir'] > 270), data['Dir_std_1']-180, data['Dir_std_1'])

## Distance to QB

In [118]:
track_qb = tracking[tracking['pff_positionLinedUp']=='QB'][['gameId','playId','frameId','x','y']]
data = data.merge(track_qb, on=['gameId','playId','frameId'], suffixes=['','_qb'], how='left')
# sanity check (1824123)
data.shape

(1824123, 89)

In [119]:
data['dist_qb'] = np.sqrt((data['x'] - data['x_qb'])**2 + (data['y'] - data['y_qb'])**2)
data.head()

,Unnamed: 0,Unnamed: 0.1,index,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,position,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,isOnOffense,ToLeft,X_std,Y_std,back,left,right,distance_blocker,distance_rusher,pressure,pressure_allowed,edge,interiorDL,LB,T,G,C,interiorOL,num_rushers,los_snap,dist_los_snap,O_std_1,O_std_2,Dir_std_1,Dir_std_2,x_qb,y_qb,dist_qb
0,0,177,177,2021090900,97,40151.0,6,2021-09-10T00:26:31.600,66.0,TB,right,42.10,24.02,0.51,1.98,0.06,48.28,308.34,ball_snap,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.10,24.02,36.56,30.92,16.92,NaN,1.705315,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54,48.28,48.28,308.34,308.34,37.64,24.26,4.466453
1,1,178,178,2021090900,97,40151.0,7,2021-09-10T00:26:31.700,66.0,TB,right,42.03,24.06,0.79,2.13,0.07,53.21,305.04,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,42.03,24.06,36.56,30.92,16.92,NaN,1.691774,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54,53.21,53.21,305.04,305.04,37.56,24.26,4.474472
2,2,179,179,2021090900,97,40151.0,8,2021-09-10T00:26:31.800,66.0,TB,right,41.95,24.10,1.00,1.84,0.09,57.07,300.02,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.95,24.10,36.56,30.92,16.92,NaN,1.647695,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54,57.07,57.07,300.02,300.02,37.47,24.25,4.482510
3,3,180,180,2021090900,97,40151.0,9,2021-09-10T00:26:31.900,66.0,TB,right,41.86,24.16,1.17,1.35,0.11,59.92,301.28,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.86,24.16,36.56,30.92,16.92,NaN,1.569841,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54,59.92,59.92,301.28,301.28,37.38,24.24,4.480714
4,4,181,181,2021090900,97,40151.0,10,2021-09-10T00:26:32.000,66.0,TB,right,41.74,24.24,1.39,1.09,0.14,62.96,301.82,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,C,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,True,False,41.74,24.24,36.56,30.92,16.92,NaN,1.478276,0,0,0,0,0,0,0,1,1,5.0,41.56,-0.54,62.96,62.96,301.82,301.82,37.27,24.23,4.470011


In [120]:
data[data['pff_role']=='Pass Rush'].to_csv('C:/Users/19012/Documents/BDB 2023/pass_rushers.csv')
data[data['pff_role']=='Pass Block'].to_csv('C:/Users/19012/Documents/BDB 2023/pass_blockers.csv')